In [355]:
# -*- coding: utf-8 -*-
"""
   Introduction to Deep Learning (LDA-T3114)
   Skeleton code for Assignment 1: Training Linear Classifiers

   Hande Celikkanat & Miikka Silfverberg
"""

'\n   Introduction to Deep Learning (LDA-T3114)\n   Skeleton code for Assignment 1: Training Linear Classifiers\n\n   Hande Celikkanat & Miikka Silfverberg\n'

In [346]:
%matplotlib inline

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

#ATTENTION: If necessary, add the paths to your data_semeval.py and paths.py here:
#import sys
#sys.path.append('</path/to/below/modules>')
from data_semeval import *
from paths import data_dir

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ollijokinen/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
#--- hyperparameters ---

n_classes = len(LABEL_INDICES)
n_epochs = 30
learning_rate = 0.05
report_every = 1
verbose = False

In [4]:
def make_bow(tweet, indices):
    feature_ids = list(indices[tok] for tok in tweet['BODY'] if tok in indices)
    bow_vec = torch.zeros(len(indices))
    bow_vec[feature_ids] = 1
    return bow_vec.view(1, -1)

def generate_bow_representations(data):
    vocab = set(token for tweet in data['training'] for token in tweet['BODY'])
    vocab_size = len(vocab) 
    indices = {w:i for i, w in enumerate(vocab)}
  
    for split in ["training","development.input","development.gold","test.input","test.gold"]:
        for tweet in data[split]:
            tweet['BOW'] = make_bow(tweet,indices)

    return indices, vocab_size

# Convert string label to pytorch format.
def label_to_idx(label):
    return torch.LongTensor([LABEL_INDICES[label]])

In [5]:
#--- model ---
class LogisticRegression(nn.Module):
    def __init__(self, vocab_size, n_classes):
        super(LogisticRegression, self).__init__()
        # WRITE CODE HERE
        self.linear = torch.nn.Linear(vocab_size,n_classes)
        

    def forward(self, input):
      # WRITE CODE HERE
        output = F.log_softmax(self.linear(input))
        return output
        
        

In [6]:
#--- data loading ---
data = read_semeval_datasets(data_dir)
indices, vocab_size = generate_bow_representations(data)

In [7]:
#--- set up ---
model = LogisticRegression(vocab_size, n_classes)

# WRITE CODE HERE
criterion = torch.nn.NLLLoss()
optimizer = optim.SGD(model.parameters(),lr=learning_rate)

22031

In [12]:
#--- training ---
for epoch in range(n_epochs):
    total_loss = 0
    for tweet in data['training']:  
        gold_class = label_to_idx(tweet['SENTIMENT'])
        #WRITE CODE HERE   
        x = tweet["BOW"]
        optimizer.zero_grad() 
        outputs = model(x) 
        loss = criterion(outputs, gold_class) 
        loss.backward() 
        optimizer.step() 
        total_loss += loss
        
    if ((epoch+1) % report_every) == 0:
        print('epoch: %d, loss: %.4f' % (epoch, total_loss/len(data['training'])))
    
    

/Users/ollijokinen/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  # This is added back by InteractiveShellApp.init_path()


epoch: 0, loss: 0.8298
epoch: 1, loss: 0.5866
epoch: 2, loss: 0.4695
epoch: 3, loss: 0.3927
epoch: 4, loss: 0.3374
epoch: 5, loss: 0.2955
epoch: 6, loss: 0.2625
epoch: 7, loss: 0.2360
epoch: 8, loss: 0.2142
epoch: 9, loss: 0.1959
epoch: 10, loss: 0.1804
epoch: 11, loss: 0.1671
epoch: 12, loss: 0.1555
epoch: 13, loss: 0.1454
epoch: 14, loss: 0.1364
epoch: 15, loss: 0.1285
epoch: 16, loss: 0.1214
epoch: 17, loss: 0.1150
epoch: 18, loss: 0.1092
epoch: 19, loss: 0.1040
epoch: 20, loss: 0.0992
epoch: 21, loss: 0.0949
epoch: 22, loss: 0.0909
epoch: 23, loss: 0.0872
epoch: 24, loss: 0.0838
epoch: 25, loss: 0.0807
epoch: 26, loss: 0.0778
epoch: 27, loss: 0.0751
epoch: 28, loss: 0.0726
epoch: 29, loss: 0.0702


In [11]:
#--- test ---
correct = 0
with torch.no_grad():
    for tweet in data['test.gold']:
        gold_class = label_to_idx(tweet['SENTIMENT'])

        # WRITE CODE HERE
        y = tweet["BOW"] #tensor
        outputs = model(y)
        _,predicted = torch.max(outputs.data,1)
        correct += torch.eq(predicted,gold_class).item()
        

        if verbose:
            print('TEST DATA: %s, OUTPUT: %s, GOLD LABEL: %d' % 
                (tweet['BODY'], tweet['SENTIMENT'], predicted))

print('test accuracy: %.2f' % (100.0 * correct / len(data['test.gold'])))

/Users/ollijokinen/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  # This is added back by InteractiveShellApp.init_path()


test accuracy: 43.99


## TEST ACCURACY: 65.18